In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.compose import TransformedTargetRegressor

from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from catboost import CatBoostRegressor


In [ ]:
pip install catboost

In [ ]:
!git clone --single-branch --branch v0.2dev https://github.com/scikit-learn-contrib/py-earth.git
%cd py-earth
!python setup.py install --cythonize

In [3]:
data = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')
data.columns = [x.strip() for x in data.columns.tolist()]
X = data.drop(columns = ['weekday_is_monday',
 'weekday_is_saturday',
 'LDA_00',
 'n_unique_tokens',
 'n_non_stop_words',
 'self_reference_avg_sharess',
 'rate_positive_words',
 'LDA_04',
 'LDA_02',
 'kw_max_max',
 'kw_avg_min',
 'kw_avg_avg',
 'rate_negative_words',
 'min_negative_polarity', 'url', 'timedelta', 'shares'])
y = data['shares']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

# Random Forest Model

In [ ]:
random_forest_model = RandomForestRegressor(max_depth = 20, bootstrap = True, max_features = 10, n_estimators = 400, 
                                            oob_score = True)
random_forest_model = TransformedTargetRegressor(random_forest_model, func = np.log, 
                                                 inverse_func = np.exp).fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_test, random_forest_model.predict(X_test))

# XGBoost

In [ ]:
xgb_model = xgb.XGBRegressor(random_state = 1, max_depth = 4, learning_rate = 0.005,
                        n_estimators = 1000,reg_lambda = 0, reg_alpha = 3, gamma = 6, subsample = 0.65, 
                         colsample_bytree = 0.7, eval_metric = mean_absolute_error)
xgb_model = TransformedTargetRegressor(xgb_model, func = np.log, inverse_func = np.exp)

xgb_model.fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_test, xgb_model.predict(X_test))

# Mars

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from pyearth import Earth

# Define the model
mars_model = Earth()

# Define the parameters
parameters = {'max_terms': [300,500,1000],
              'max_degree': [1,2,3,4]}

# Define the cross-validator
CV = KFold(n_splits = 5, shuffle=True, random_state=1)

# Initialize the GridSearchCV object
model = GridSearchCV(estimator=mars_model, 
                     param_grid=parameters, 
                     cv=CV, 
                     n_jobs=-1, 
                     verbose=1, 
                     scoring='neg_mean_absolute_error')

# Fit the GridSearchCV object to the data
model.fit(X_train, y_train)  

# Print the best score and best parameters
print('Best score:', model.best_score_)
print('Best parameters:', model.best_params_)


In [ ]:
mars_model = Earth(max_terms = 500, max_degree = 2)
mars_model = TransformedTargetRegressor(mars_model, func = np.log, inverse_func = np.exp).fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_train, mars_model.predict(X_train))

In [ ]:
mean_absolute_error(y_test, mars_model.predict(X_test))

## CatBoostRegressor



In [ ]:
X_train.shape

In [ ]:
#K-fold cross validation to find optimal parameters for CatBoost regressor
start_time = time.time()
param_grid = {'max_depth': [4,6,8],
              'num_leaves': [20, 31, 40],
              'learning_rate': [0.01, 0.05, 0.1],
               'reg_lambda':[0, 10, 100],
                'n_estimators':[100, 500, 1000],
                'subsample': [0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=CatBoostRegressor(random_state=1, verbose=False),                                                       
                             param_distributions = param_grid, n_iter = 200,
                             verbose = 1,random_state = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_train,y_train)
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
645 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
645 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5734, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2341, in _fit
    train_

Optimal parameter values = {'subsample': 0.75, 'reg_lambda': 10, 'num_leaves': 31, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.01}
Optimal cross validation R-squared =  0.02196835563330628
Time taken =  27  minutes


In [ ]:
# Instantiate the model with the provided parameters
cat_model = CatBoostRegressor(subsample=0.75, 
                              reg_lambda=10, 
                              depth=6, 
                              n_estimators=1000, 
                              learning_rate=0.01,
                              random_state=1, 
                              verbose=False)

# Fit the model to your training data
cat_model= TransformedTargetRegressor(cat_model, func = np.log, inverse_func = np.exp).fit(X_train, y_train)

# Predict on your test set
y_pred = cat_model.predict(X_test)  

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)  

print('Mean Absolute Error:', mae)

## Ensemble Model

In [ ]:
#Voting ensemble: Averaging the predictions of all models
en=VotingRegressor(estimators = [('xgb',xgb_model),('random_forest',random_forest_model),('mars',mars_model), ])
en.fit(X_train,y_train)
print("Ensemble model MAE = ", mean_absolute_error(en.predict(X_test),y_test))

In [ ]:
#Stacking using LinearRegression as the metamodel
en2 = StackingRegressor(estimators = [('xgb', xgb_model),('random_forest', random_forest_model),('mars', mars_model),('cat',cat_model)],
                     final_estimator=LinearRegression(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en2.fit(X_train,y_train)


In [ ]:
print("Linear regression metamodel MAE = ", mean_absolute_error(en2.predict(X_test),y_test))

Linear regression metamodel MAE =  2854.33881663202


In [ ]:
#Stacking using Lasso as the metamodel
en3 = StackingRegressor(estimators = [('xgb', xgb_model),('random_forest', random_forest_model),('mars', mars_model),('cat',cat_model)],
                     final_estimator=LassoCV(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en3.fit(X_train,y_train)


StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('xgb',
                               TransformedTargetRegressor(func=<ufunc 'log'>,
                                                          inverse_func=<ufunc 'exp'>,
                                                          regressor=XGBRegressor(base_score=None,
                                                                                 booster=None,
                                                                                 callbacks=None,
                                                                                 colsample_bylevel=None,
                                                                                 colsample_bynode=None,
                                                                                 colsample_bytree=0.5,
                                                                                 early_stopping_rounds=None,
                                                                                 enable_categorical=False,
                                                                                 eval_met...
                                                          regressor=RandomForestRegressor(max_depth=20,
                                                                                          max_features=10,
                                                                                          n_estimators=400,
                                                                                          oob_score=True))),
                              ('mars',
                               TransformedTargetRegressor(func=<ufunc 'log'>,
                                                          inverse_func=<ufunc 'exp'>,
                                                          regressor=Earth(max_degree=2,
                                                                          max_terms=500))),
                              ('cat',
                               TransformedTargetRegressor(func=<ufunc 'log'>,
                                                          inverse_func=<ufunc 'exp'>,
                                                          regressor=<catboost.core.CatBoostRegressor object at 0x7ff81dd49b40>))],
                  final_estimator=LassoCV())

In [ ]:
print("Lasso metamodel RMSE = ", mean_absolute_error(en3.predict(X_test),y_test))

Lasso metamodel RMSE =  2850.4961450500055
